In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../dataset/diagnosed_cbc_data_v4.csv')

Export du jeu de données

In [4]:
df.to_csv("../dataset/data_cleaned.csv", index=False)